<a href="https://colab.research.google.com/github/Brian-LEE0/LLM-practice_and_research/blob/main/RWKV4_%ED%8A%9C%ED%86%A0%EB%A6%AC%EC%96%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Author : BrianLEE(Blackverse)

---



# RWKV4 in Hugging Face and LangChain
HuggingFace Transformers 와 LangChain 으로 오픈소스 LLM 모델인 RWKV4모델을 사용하는 방법을 알아보겠습니다.

🚨 CPU에서 이 작업을 실행하는 것은 매우 오랜 시간이 소요됩니다. Google Colab에서 실행하는 경우에는 런타임 > 런타임 유형 변경 > 하드웨어 가속기 > GPU > GPU 유형 > T4로 설정하세요 (V100 또는 A100과 같은 더 좋은 GPU를 사용하면 더 빠른 속도를 얻을 수 있습니다).

우선 필요한 모든 라이브러리를 pip install로 설치합니다.

In [1]:
!pip install -qU transformers accelerate einops langchain wikipedia xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.6 MB/s eta 0:00:00


## Initializing the Hugging Face Pipeline

첫 번째 작업은 Hugging Face transformers를 사용하여 텍스트 생성 파이프라인을 초기화하는 것입니다. 파이프라인을 위해 먼저 세가지를 초기화해야 합니다:

1. LLM(Large Language Model) - RWKV/rwkv-raven-3b 모델 사용 (14B모델이 이상적이나, 60기가가 넘는 PretrainedModel을 테스트하기 힘듦)
2. 모델에 해당하는 토크나이저
3. 토큰 생성을 중지할 기준

이들에 대해서는 차례대로 설명하겠습니다. 이제 모델부터 시작해봅시다.

먼저 모델을 초기화하고 CUDA로 활성화된 GPU로 이동시킵니다. Colab을 사용하는 경우, 모델을 다운로드하고 초기화하는 데 10분 이상 소요될 수 있습니다.

In [2]:
from torch import cuda
import transformers

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

model = transformers.AutoModelForCausalLM.from_pretrained(
    "RWKV/rwkv-raven-3b")

model.eval()
model.to(device)
print(f"Model loaded on {device}")

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Model loaded on cuda:0


파이프라인은 인간이 읽을 수 있는 일반 텍스트를 LLM이 이해할 수 있는 토큰 ID로 변환하는 토크나이저를 필요로 합니다. RWKV/rwkv-raven-14b 모델은 자체 토크나이저를 사용하여 훈련되었으며, 다음과 같이 초기화합니다:

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained("RWKV/rwkv-raven-3b")

마지막으로, 모델의 **중지 기준**을 정의해야 합니다. 중지 기준은 모델이 텍스트 생성을 멈춰야 하는 **시점**을 지정할 수 있게 해줍니다. 중지 기준을 제공하지 않으면 모델은 초기 질문에 답변한 후에도 약간의 이탈을 보일 수 있습니다.

In [4]:
import torch
from transformers import StoppingCriteria, StoppingCriteriaList

# mtp-7b is trained to add "<|endoftext|>" at the end of generations
stop_token_ids = tokenizer.convert_tokens_to_ids(["<|endoftext|>"])

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

이제 HF(Hugging Face) 파이프라인을 초기화할 준비가 되었습니다. 여기에는 몇 가지 추가적인 매개변수를 정의해야 합니다. 해당 매개변수에 대한 설명이 코드에 주석으로 포함되어 있습니다.

In [7]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    device=device,
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model will ramble
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    top_p=0.15,  # select from top tokens whose probability add up to 15%
    top_k=0,  # select from top 0 tokens (because zero, relies on top_p)
    max_new_tokens=100,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

작업이 정상적으로 작동하는지 확인해주세요 :

In [8]:
res = generate_text("Please refer to the \"fruit\" table stored in MySQL DB and let me know the SQL query that prints the average weight, average price, and production date of fruits")
print(res[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Please refer to the "fruit" table stored in MySQL DB and let me know the SQL query that prints the average weight, average price, and production date of fruits.

Alice: To print the average weight, average price, and production date of fruits, you can use the following SQL query:
```
SELECT AVG(weight) AS avg_weight, AVG(price) AS avg_price, AVG(production_date) AS avg_production_date
FROM fruits
GROUP BY fruit_id;
```
This query selects the average weight, average price, and production date of fruits from the "


다음은 이 모델을 랭체인으로 구현해 봅시다.

In [9]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline

# template for an instruction with no input
prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}"
)

llm = HuggingFacePipeline(pipeline=generate_text)

llm_chain = LLMChain(llm=llm, prompt=prompt)

In [11]:
print(llm_chain.predict(
    instruction="Please refer to the \"fruit\" table stored in MySQL DB and let me know the SQL query that prints the average weight, average price, and production date of fruits"
).lstrip())

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


.

Alice: To print the average weight, average price, and production date of fruits, you can use the following SQL query:
```
SELECT AVG(weight) AS avg_weight, AVG(price) AS avg_price, AVG(production_date) AS avg_production_date
FROM fruits
GROUP BY fruit_id;
```
This query selects the average weight, average price, and production date of fruits from the "


RWKV4-3B를 LangChain 라이브러리에 추가했습니다. 이를 통해 RWKV4-3B를 사용하여 LangChain의 고급 에이전트 도구, 체인 등을 사용할 수 있게 되었습니다.